<a href="https://colab.research.google.com/github/SkdooshNB/rice-leaf-disease-detection/blob/master/fastAPI_calls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd drive/MyDrive/plant_disease/saved_model/

/content/drive/MyDrive/plant_disease/saved_model


In [ ]:
# !pip install colabcode fastapi nest-asyncio pyngrok uvicorn python-multipart
# !pip install fastapi nest-asyncio pyngrok uvicorn python-multipart
!pip install FastAPI -q
!pip install uvicorn -q
!pip install pyngrok python-multipart nest-asyncio


In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers

In [ ]:
model = tf.keras.models.load_model('1')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)       

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
  np_image = Image.open(filename)
  # np_image = np.array(np_image).astype('uint8')/255
  # np_image = transform.resize(np_image,(256,256,3))
  np_image = np.expand_dims(np_image, axis = 0)
  return np_image


In [ ]:
# # [[256,256,3]]
# image = load('fabf3179-8fba-4551-93fe-28e7f6c0efc3___RS_LB 3269.JPG')
# prediction = model.predict(image)
# prediction

In [ ]:
from fastapi import FastAPI,File,UploadFile
from fastapi.middleware.cors import CORSMiddleware

import nest_asyncio
from pyngrok import ngrok
import uvicorn

import numpy as np
import tensorflow as tf
from io import BytesIO
from PIL import Image

app = FastAPI()

CLASS_NAMES = ["Early Blight", "Late Blight", "Healthy"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
def ping():
  return "hello, I am alive"

def read_file_as_image(data) -> np.ndarray:
  image = np.array(Image.open(BytesIO(data)))
  return image

@app.post("/")
async def predict(file: UploadFile = File(...)):
  image = read_file_as_image(await file.read())
  img_batch = np.expand_dims(image,0)

  predictions = model.predict(img_batch)

  predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
  confidence = np.max(predictions[0])
  return {
      'class' : predicted_class,
      'confidence': float(confidence)
  }


ngrok_tunnel = ngrok.connect(3000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app,port=3000)


Public URL: http://4cbd-35-245-171-18.ngrok.io


INFO:     Started server process [341]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:3000 (Press CTRL+C to quit)


INFO:     2409:4062:286:a701:64e1:448:ffff:c855:0 - "GET / HTTP/1.1" 200 OK
INFO:     157.41.174.86:0 - "POST / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [341]
